In [1]:
#from selenium import webdriver
#from selenium.webdriver import Chrome
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.support.ui import WebDriverWait as wait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.common.exceptions import NoSuchElementException
# from selenium.webdriver.common.by import By
#from tqdm import tqdm

In [2]:
import requests 
from bs4 import BeautifulSoup
import re
import pandas as pd
from time import sleep
from selenium import webdriver
from selenium.webdriver import Chrome

In [3]:
#browser = webdriver.Chrome()

In [4]:
data=[]                                                    # создаём пустой список, в который будут собираться объявления

for i in range(1, 51):                                     # задаем цикл с номерами страниц для загрузки
    browser = webdriver.Chrome()
    print(i)                                               # выводим на печать номер обрабатываемой страницы
    url = f'https://www.list.am/ru/category/56/{i}?type=1&n=1&_a39=2&_a3_1=60&_a3_2=100' # задаем url
    browser.get(url)
    sleep(7)
    soup = BeautifulSoup(browser.page_source, 'lxml')     # Преобразуем дынные для дальнейшей работы
    
    flats = soup.findAll('div', class_='dl')[1].find('div', class_='gl').findAll('a') # находим все объявления на странице
    
    for flat in flats:                                      # проходим по списку с объявлениями и извлекаем информацию о каждом
        
        try:
            
            link = 'https://www.list.am' + flat.get('href')  # извлекаем ссылку на объявление
            price = flat.find('div', class_='p').text        # извлекаем цену на квартиру
            information = flat.find('div', class_='at').text # извлекаем короткое описание квартиры
           
            result = re.findall(r'\d{1,3}', information) # Создаем промежуточную переменную с площадью и кол-ом комнат квартиры
            
            number = re.findall(r'\d+', link)[0]              # извлекаем номер объявления
            region = re.findall(r'\w+,', information)[0][:-1] # извлекаем регион
            rooms = int(result[0])                            # извлекаем количество комнат
            square = int(result[1])                           # извлекаем площадь
            value = re.findall(r'[\֏\$]', price)[0]           # извлекаем валюту цены
            cost = int(re.findall(r'\d+[,]?\d+[,]?\d+', price)[0].replace(',','')) # извлекаем цену
            
            if value == '$':                                  # если цена в долларах, конвертируем её в драммы
                cost *= 390
                value = '֏'

            data.append([number, region, cost, value, rooms, square, price+', ' +information, link]) # добавляем всю информацию

        except:
            print("Исключение было обработано") 
    browser.quit()

print('THE END')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
Исключение было обработано
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
Исключение было обработано
49
Исключение было обработано
Исключение было обработано
50
Исключение было обработано
THE END


In [4]:
# browser.find_element(By.CSS_SELECTOR, ".dlf>a").click()

In [5]:
headers = ['number', 'region', 'cost', 'value', 'rooms', 'square', 'information', 'link'] # задаем названия колонок
df = pd.DataFrame(data, columns=headers)                                                  # создаем DataFrame
df = df.drop_duplicates()

In [6]:
df.to_csv(r'D:\Ela\flats_Yerevan\flats_yerevan.csv', index=False, sep=';', encoding='utf8') # сохраняем в файл

In [7]:
df

,number,region,cost,value,rooms,square,information,link
0,18736872,Кентрон,280000,֏,2,60,"280,000 ֏ в месяц, Кентрон, 2 ком., 60 кв.м., ...",https://www.list.am/ru/item/18736872
1,19502269,Кентрон,360000,֏,2,95,"360,000 ֏ в месяц, Кентрон, 2 ком., 95 кв.м., ...",https://www.list.am/ru/item/19502269
2,21503687,Кентрон,585000,֏,2,70,"$1,500 в месяц, Кентрон, 2 ком., 70 кв.м., 5/5...",https://www.list.am/ru/item/21503687
3,19403448,Арабкир,385000,֏,2,100,"385,000 ֏ в месяц, Арабкир, 2 ком., 100 кв.м.,...",https://www.list.am/ru/item/19403448
4,20070328,Канакер,350000,֏,3,70,"350,000 ֏ в месяц, Зейтун Канакер, 3 ком., 70 ...",https://www.list.am/ru/item/20070328
...,...,...,...,...,...,...,...,...
4790,16198939,Кентрон,390000,֏,2,65,"$1,000 в месяц, Кентрон, 2 ком., 65 кв.м., 5/5...",https://www.list.am/ru/item/16198939
4791,16881964,Кентрон,468000,֏,3,80,"$1,200 в месяц, Кентрон, 3 ком., 80 кв.м., 9/1...",https://www.list.am/ru/item/16881964
4792,16136164,Кентрон,546000,֏,3,100,"$1,400 в месяц, Кентрон, 3 ком., 100 кв.м., 10...",https://www.list.am/ru/item/16136164
4793,17342111,Кентрон,650000,֏,3,78,"650,000 ֏ в месяц, Кентрон, 3 ком., 78 кв.м., ...",https://www.list.am/ru/item/17342111
